In [ ]:
import obi_one as obi
import os
import pandas as pd

In [ ]:
input_root = "/Users/pokorny/Data/Circuits"
output_root = "../../../obi-output/extracted_small_microcircuits"

In [ ]:
# # Neuron IDs for nbS1-HEX0
# N_ids = 10
# node_selection_files = ["./selection1.csv",
#                         "./selection2.csv"]
# def sel_first_n(elements, n):
#     assert len(elements) >= n, f"ERROR: Less than {n} elements to select from!"
#     return elements[:n]
# node_id_lists = [obi.NamedTuple(name=f"N_{N_ids}__" + os.path.splitext(os.path.split(_f)[1])[0], elements=tuple(sel_first_n(pd.read_csv(_f)["node_ids"].to_list(), N_ids))) for _f in node_selection_files]

In [ ]:
# Neuron IDs for rCA1-CYLINDER-REF
node_id_lists = [#obi.NamedTuple(name="N_2__rCA1-FFInh-Sel1", elements=(1146, 5684)),  # + virtual 64063
                 # obi.NamedTuple(name="N_3__rCA1-ThetaOsc-Sel1", elements=(3903, 5712, 6026)),
                 # obi.NamedTuple(name="N_10__rCA1-ThetaOsc-Sel1", elements=(2009, 3371, 1956, 3673, 1464, 4806, 2197, 1147, 5702, 6026)),
                 # obi.NamedTuple(name="N_8__rCA1-ThetaOsc-Sel1", elements=(5842, 5702, 5850, 5628,  # PV+
                 #                                                          2886, 3049, 3673, 4806)),  # PC
                 obi.NamedTuple(name="N_9__rCA1-ThetaOsc-Sel1", elements=(5625, 5821, 5847, 5815, 5591, 5619, 5675, 5845,  # PV+
                                                                          3673)),  # PC
                ]

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        # circuit=obi.Circuit(name="ToyCircuit-S1-6k", path=os.path.join(input_root, "ToyCircuit-S1-6k", "circuit_config.json")),
                        # circuit=obi.Circuit(name="nbS1-HEX0", path=os.path.join(input_root, "nbS1-HEX0", "circuit_config.json")),
                        circuit=obi.Circuit(name="rCA1-CYLINDER-REF", path=os.path.join(input_root, "rCA1-CYLINDER-REF", "circuit_config_postfix2.json")),
                        run_validation=False,
                        do_virtual=True,
                        create_external=True,
                        virtual_sources_to_ignore=("external_hippocampus_neurons__hippocampus_neurons__chemical_synapse", ),
                    ),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Excitatory", "Inhibitory"], random_sample=[100, 500]),
                    # neuron_set=obi.PredefinedNeuronSet(node_set=["Layer2", "Layer3", "Layer4", "Layer5", "Layer6"], random_sample=10),
                    neuron_set=obi.IDNeuronSet(neuron_ids=node_id_lists)
)

grid_scan = obi.GridScanGenerationTask(form=circuit_extractions_scan_config, output_root=output_root, coordinate_directory_option="VALUE")

In [ ]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)

In [ ]:
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

In [ ]:
# Check circuits
from bluepysnap import Circuit

for _inst in grid_scan.coordinate_instances():
    cfg = grid_scan.output_root / str(_inst.neuron_set.neuron_ids) / "circuit_config.json"
    c = Circuit(cfg)
    # print(f"{_inst.neuron_set.neuron_ids}: {c.nodes['S1nonbarrel_neurons'].size} neurons, {c.edges["S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].size} synapses")
    print(f"{_inst.neuron_set.neuron_ids}: {c.nodes['hippocampus_neurons'].size} neurons, {c.edges["hippocampus_neurons__hippocampus_neurons__chemical_synapse"].size} synapses")
